In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import os

## 1. Data Loading and Merging

We'll start by loading our two datasets:
1. `customer_social_profiles.csv`: Contains social media engagement data
2. `customer_transactions.csv`: Contains transaction history

We'll need to:
1. Load both datasets
2. Clean and preprocess the data
3. Merge the datasets based on customer ID
4. Perform feature engineering

In [3]:
# Load the datasets
social_profiles = pd.read_csv('../data/tables/customer_social_profiles.csv')
transactions = pd.read_csv('../data/tables/customer_transactions.csv')

# Display basic information about the datasets
print("Customer Social Profiles Dataset:")
print(social_profiles.info())
print("\nCustomer Transactions Dataset:")
print(transactions.info())

Customer Social Profiles Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id_new          155 non-null    object 
 1   social_media_platform    155 non-null    object 
 2   engagement_score         155 non-null    int64  
 3   purchase_interest_score  155 non-null    float64
 4   review_sentiment         155 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 6.2+ KB
None

Customer Transactions Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id_legacy  150 non-null    int64  
 1   transaction_id      150 non-null    int64  
 2   purchase_amount     150 non-null    int64  
 3   purchase_date       150 non-nul

In [ ]:
# Clean and preprocess data
# Convert customer IDs to a common format (removing 'A' prefix from social profiles)
social_profiles['customer_id'] = social_profiles['customer_id_new'].str[1:].astype(int)
transactions['customer_id'] = transactions['customer_id_legacy'].astype(int)

# Merge datasets
merged_data = pd.merge(
    social_profiles,
    transactions,
    on='customer_id',
    how='inner'
)

# Feature engineering
# Convert date to datetime
merged_data['purchase_date'] = pd.to_datetime(merged_data['purchase_date'])

# Create additional features
merged_data['engagement_purchase_ratio'] = merged_data['engagement_score'] / merged_data['purchase_amount']
merged_data['sentiment_numeric'] = merged_data['review_sentiment'].map({'Positive': 1, 'Neutral': 0, 'Negative': -1})

# Display merged dataset info
print("Merged Dataset Information:")
print(merged_data.info())

# Fill missing values 
for col in merged_data.columns:
    if merged_data[col].dtype in ['int64', 'float64']:
        # Use median for robustness (handles skewed distributions better)
        merged_data[col].fillna(merged_data[col].median(), inplace=True)
    else:
        # For categorical columns, fill with mode (most common value)
        merged_data[col].fillna(merged_data[col].mode()[0], inplace=True)

# Save merged dataset
merged_data.to_csv('../data/tables/merged_customer_data.csv', index=False)
print("\nMerged dataset saved to tables folder")
